In [ ]:
import tensorflow as tf
print(tf.__version__)
from music21 import converter, instrument, note, chord, duration
import glob
import pickle
import numpy as np
from music21 import converter, instrument, note, chord, stream
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Dropout
from keras.layers import LSTM
from keras.layers import Activation, Lambda
from keras.layers import BatchNormalization as BatchNorm
from keras.utils import np_utils
from keras.callbacks import ModelCheckpoint
from keras.models import load_model
from collections import OrderedDict

In [ ]:
def create_network(network_input, n_vocab, temperature):
    '''
    defines function w/ that takes three arguments:
      input data,
      # of unique elements/size of output layer,
      scaling factor controlling randomness of networks predictions
    '''

    model = Sequential()
    model.add(LSTM(
        512,
        input_shape=(network_input.shape[1], network_input.shape[2]),
        recurrent_dropout=0.3,
        return_sequences=True
    ))

    model.add(LSTM(512, recurrent_dropout=0.3))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(256))
    model.add(Activation('relu'))
    model.add(BatchNorm())
    model.add(Dropout(0.3))
    model.add(Dense(n_vocab))
    model.add(Lambda(lambda x: x/temperature))
    model.add(Activation('softmax'))

    model.compile(loss='categorical_crossentropy', optimizer='adam')

    return model

def get_list_of_all_possible_notes(glob_query):
    '''
    Cycles through all MIDI files using the given glob_query, collecting every possible note (consisting of pitch and duration) found in each file.
    Creates an unordered list of the SET of these notes, dumps them into pickle file notes.p, and returns the list of notes.
    '''
    notes = set()
    for file in glob.glob(glob_query):
        midi = converter.parse(file)
        print("Parsing %s" % file)
        notes_to_parse = None
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                note_str = str(element.duration) + "-" + str(element.pitch)
                notes.add(note_str)
            elif isinstance(element, chord.Chord):
                chord_str = str(element.duration) + "-" + '.'.join(str(n) for n in element.normalOrder)
                notes.add(chord_str)
            elif isinstance(element, note.Rest):
                notes.add("rest"+"-"+str(element.duration))


    note_list = list(notes)
    pickle.dump(note_list, open('notes.p', 'wb'))

    return note_list


def get_sequences(glob_query):
    '''
    Cycles through all MIDI files using the given glob_query, collecting every possible note (consisting of pitch and duration) found in each file.
    Creates a list of these notes in their sequence order, and returns this list.
    The output sequences are in "human" form (i.e. note name, duration)
    '''
    sequences = []
    for file in glob.glob(glob_query):
        midi = converter.parse(file)
        notes_to_parse = None
        try:
            s2 = instrument.partitionByInstrument(midi)
            notes_to_parse = s2.parts[0].recurse()
        except:
            notes_to_parse = midi.flat.notes
        sequence = []
        for element in notes_to_parse:
            if isinstance(element, note.Note):
                note_str = str(element.duration) + "-" + str(element.pitch)
                sequence.append(note_str)
            elif isinstance(element, chord.Chord):
                chord_str = str(element.duration) + "-" + '.'.join(str(n) for n in element.normalOrder)
                sequence.append(chord_str)
            elif isinstance(element, note.Rest):
                sequence.append("rest"+"-"+str(element.duration))
        sequences.append(sequence)

    return sequences

def prepare_sequences(sequences):
    '''
    Prepare the sequences used by the Neural Network.
    Convert the sequences from human-readable form to integer form (numeric index) and one-hot encode them.
    Returns the input sequences and generated output.
    '''
    input_sequence_length = 8

    # Form "note_to_int" lookup dictionary using notes.p
    pitchnames = pickle.load(open("notes.p", "rb"))
    note_to_int = dict((note, number) for number, note in enumerate(pitchnames))
    n_vocab = len(pitchnames)

    print("Decoding Dictionary:")
    for key, value in note_to_int.items():
        print(f"{key}: {value}")


    network_input = []
    network_output = []

    for sequence in sequences:
        for i in range(len(sequence) - input_sequence_length):
            sequence_in = sequence[i:i + input_sequence_length]
            sequence_out = sequence[i + input_sequence_length]

            network_input.append([note_to_int[char] for char in sequence_in])
            network_output.append(note_to_int[sequence_out])

    n_patterns = len(network_input)

    # Reshape the input into a format compatible with LSTM layers
    network_input = np.reshape(network_input, (n_patterns, input_sequence_length, 1))
    network_input = np_utils.to_categorical(network_input, num_classes=n_vocab)
    network_output = np_utils.to_categorical(network_output, num_classes=n_vocab)

    return network_input, network_output

vocabulary = get_list_of_all_possible_notes("/content/Jazz/*.mid")
sequence = get_sequences("/content/Jazz/*.mid")
network_input, network_output = prepare_sequences(sequence)

def train_network(training_sequence, vocabulary):
    """ Train a Neural Network to generate music """

    if not training_sequence:
        print("Error: List of notes is empty")
        return

    n_vocab = len(vocabulary) #calculate the number of unique notes

    network_input, network_output = prepare_sequences(training_sequence)

    model = create_network(network_input, n_vocab, 1)


    checkpoint = ModelCheckpoint(
        "bestweights.hdf5",
        monitor='loss',
        verbose=0,
        save_best_only=True, #saving the best weights
        mode='min'#saving weights with the smallest value of the loss function
    )

    callbacks_list = [checkpoint]

    # Train the model with the prepared data and callbacks
    model.fit(network_input, network_output, epochs=500, batch_size=64, callbacks=callbacks_list)

train_network(sequence, vocabulary)

In [ ]:
from numpy.random import choice

def get_seed_sequence(midi_file):
    """ Taking midi file as input to extract 16 note seed sequence and parses via the file
        to convert notes/chords into string representation of duration and pitch """
    midi = converter.parse(midi_file)
    notes_to_parse = None
    try:
        s2 = instrument.partitionByInstrument(midi)
        notes_to_parse = s2.parts[0].recurse()
    except:
        notes_to_parse = midi.flat.notes
    seed_sequence = []
    num_notes = 0  # Track the number of notes added to the seed sequence
    for element in notes_to_parse:
        if isinstance(element, note.Note):
            note_str = str(element.duration.quarterLength) + "-" + str(element.pitch.midi)
            seed_sequence.append(note_str)
            num_notes += 1
        elif isinstance(element, chord.Chord):
            chord_str = str(element.duration.quarterLength) + "-" + '.'.join(str(n.midi) for n in element.pitches)
            seed_sequence.append(chord_str)
            num_notes += 1
        elif isinstance(element, note.Rest):
            seed_sequence.append("rest"+"-"+str(element.duration))
            num_notes += 1
        if num_notes == 16:  # Stop after 16 notes have been added
            break
    return seed_sequence

def prepare_sequences_prediction(vocabulary, sequence):
    n_vocab = len(vocabulary)
    note_to_int = dict((note, number) for number, note in enumerate(vocabulary))

    sequence_length = 8
    network_input = []
    sequence_in = sequence[0][:sequence_length]
    network_input.append([note_to_int.get(char, 0) for char in sequence_in])

    n_patterns = len(network_input)
    categorical_input = np_utils.to_categorical(np.reshape(network_input, (n_patterns, sequence_length, 1)), num_classes=n_vocab)

    return categorical_input


def generate_notes(model, network_input, vocabulary): #model is NN, network input_ is a list of sequences of musical notes, pitchnames are all possible otes, n_vocab is number of unique musical notes
    """ Generate notes from the neural network based on a sequence of notes """
    # Starts the melody by picking a random sequence from the input as a starting point
    #start = np.random.randint(0, len(network_input)-1) #randomly selecting a starting point to generate the melody and uses sequence to predict next note

    n_vocab = len(vocabulary)
    int_to_note = dict((number, note) for number, note in enumerate(vocabulary))


    pattern = network_input[0]

    categorical_version = np.argmax(pattern, axis=1)
    prediction_output = [int_to_note[note] for note in categorical_version]

    for note_index in range(250): #process is repeated 200 times until a decent chunk  of music is composed
        prediction_input = pattern[None,:,:]
        prediction = model.predict(prediction_input, verbose=0)


        index = choice(list(range(len(prediction[0]))), 1, p=prediction[0])
        result = int_to_note[index[0]]
        prediction_output.append(result)
        pattern = np.concatenate((pattern, np_utils.to_categorical(index, num_classes = n_vocab)))
        pattern = pattern[1:len(pattern)]

    return prediction_output #returning list of generated musical notes

def generate():
    seed_sequence = get_seed_sequence("/content/Jazz Seed 3.mid")
    network_input = prepare_sequences_prediction(vocabulary, [seed_sequence])

    print(network_input)
    model = create_network(network_input, len(vocabulary), 1)
    model.load_weights('/content/bestweights.hdf5')

    prediction_output = generate_notes(model, network_input, vocabulary)
    # Print the first 16 notes of the generated output vertically
    for note in prediction_output[:16]:
        print(note)

    # Update the seed sequence with the first 16 notes of the generated output
    seed_sequence = seed_sequence[:16] + prediction_output[16:]

    create_midi(seed_sequence)

from fractions import Fraction

def create_midi(prediction_output):
    offset = 0
    output_notes = []
    for pattern_outer in prediction_output:
        pattern = pattern_outer.split("-")[1]
        duration_str = pattern_outer.split("-")[0]

        duration_parts = duration_str.split()
        if len(duration_parts) >= 2:
            duration_value = duration_parts[1][:-1]  # Extract the duration value
            if '/' in duration_value:  # Check if the duration is a fraction
                duration = float(Fraction(duration_value))  # Convert the fraction string to a float
            else:
                duration = float(duration_value)  # Convert the duration string to a float
        else:
            duration = 0.5  # Set a default duration if the value cannot be extracted

        if ('.' in pattern) or pattern.isdigit():
            notes_in_chord = pattern.split('.')
            notes = []
            for current_note in notes_in_chord:
                try:
                    new_note = note.Note(int(current_note), quarterLength=duration)
                    new_note.storedInstrument = instrument.Piano()
                    notes.append(new_note)
                except ValueError:
                    print(f"Ignoring invalid note: {current_note}")
            if notes:
                new_chord = chord.Chord(notes)
                new_chord.offset = offset
                output_notes.append(new_chord)
        else:
            try:
                new_note = note.Note(pattern, quarterLength=duration)
                new_note.offset = offset
                new_note.storedInstrument = instrument.Piano()
                output_notes.append(new_note)
            except ValueError:
                print(f"Ignoring invalid note: {pattern}")
        offset += duration

    midi_stream = stream.Stream(output_notes)
    midi_stream.write('midi', fp='test_output_beta.mid')


generate()